In [1]:
%run init.ipynb

C:\Users\Administrator\Anaconda3\lib\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.11) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.models.MatchSRNN.get_default_preprocessor()

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval:   0%| | 0/95 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.601 seconds.
Prefix dict has been built successfully.
Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 95/95 [00:02<00:00, 44.83it/s]
Processing text_right with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 97/97 [00:00<00:00, 139.85it/s]
Building Vocabulary from a datapack.: 100%|██████████████████████████████████████| 

In [4]:
preprocessor.context

{'embedding_input_dim': 328,
 'filter_unit': <mzcn.preprocessors.units.frequency_filter.FrequencyFilter at 0x1ae6483d588>,
 'vocab_size': 328,
 'vocab_unit': <mzcn.preprocessors.units.vocabulary.Vocabulary at 0x1ae723a3dd8>}

In [5]:
# glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
# term_index = preprocessor.context['vocab_unit'].state['term_index']
# embedding_matrix = glove_embedding.build_matrix(term_index)
# l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
# embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
devset = mz.dataloader.Dataset(
    data_pack=dev_pack_processed
)

In [7]:
padding_callback = mz.models.MatchSRNN.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback,
)
devloader = mz.dataloader.DataLoader(
    dataset=devset,
    stage='dev',
    callback=padding_callback,
)

In [8]:
model = mz.models.MatchSRNN()

model.params['task'] = ranking_task
# model.params['embedding'] = embedding_matrix #这里是当加载glove等模型时取消该行注释
#设置embedding系数
model.params["embedding_output_dim"]=100
model.params["embedding_input_dim"]=preprocessor.context["embedding_input_dim"]
model.params['channels'] = 4
model.params['units'] = 10
model.params['dropout'] = 0.2
model.params['direction'] = 'lt'

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

MatchSRNN(
  (embedding): Embedding(328, 100, padding_idx=0)
  (dropout): Dropout(p=0.2, inplace=False)
  (matching_tensor): MatchingTensor()
  (spatial_gru): SpatialGRU(
    (_activation): Tanh()
    (_recurrent_activation): Sigmoid()
    (_wr): Linear(in_features=34, out_features=30, bias=True)
    (_wz): Linear(in_features=34, out_features=40, bias=True)
    (_w_ij): Linear(in_features=4, out_features=10, bias=True)
    (_U): Linear(in_features=30, out_features=10, bias=False)
  )
  (out): Linear(in_features=10, out_features=1, bias=True)
)
Trainable params:  75611


In [9]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=devloader,
    validate_interval=None,
    epochs=10
)

In [10]:
trainer.run()

[Iter-1 Loss-0.939]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1538 - normalized_discounted_cumulative_gain@5(0.0): 0.1538 - mean_average_precision(0.0): 0.1538



[Iter-2 Loss-0.964]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1538 - normalized_discounted_cumulative_gain@5(0.0): 0.1538 - mean_average_precision(0.0): 0.1538



[Iter-3 Loss-0.925]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1538 - normalized_discounted_cumulative_gain@5(0.0): 0.1538 - mean_average_precision(0.0): 0.1538



[Iter-4 Loss-1.012]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1538 - normalized_discounted_cumulative_gain@5(0.0): 0.1538 - mean_average_precision(0.0): 0.1538



[Iter-5 Loss-0.944]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1538 - normalized_discounted_cumulative_gain@5(0.0): 0.1538 - mean_average_precision(0.0): 0.1538



[Iter-6 Loss-0.958]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1538 - normalized_discounted_cumulative_gain@5(0.0): 0.1538 - mean_average_precision(0.0): 0.1538



[Iter-7 Loss-0.945]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1538 - normalized_discounted_cumulative_gain@5(0.0): 0.1538 - mean_average_precision(0.0): 0.1538



[Iter-8 Loss-0.887]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1538 - normalized_discounted_cumulative_gain@5(0.0): 0.1538 - mean_average_precision(0.0): 0.1538



[Iter-9 Loss-0.898]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1538 - normalized_discounted_cumulative_gain@5(0.0): 0.1538 - mean_average_precision(0.0): 0.1538



[Iter-10 Loss-0.911]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1538 - normalized_discounted_cumulative_gain@5(0.0): 0.1538 - mean_average_precision(0.0): 0.1538

Cost time: 6.505980014801025s


In [11]:
import gc
gc.collect()

842